In [1]:
import sys
import numpy as np
import math
import time
import json
import openravepy
import pickle

sys.path.append('../../')
from src.Robots.Models import *
import Config
from prpy.planning import ompl, CBiRRTPlanner, chomp
from openravepy import planningutils
from trac_ik_python.trac_ik import IK
from src.Simulators.OpenRaveSimulator import *

In [31]:

# sim = OpenRaveSimulator(Config.OPENRAVE_ENV_XML)
# env = sim.env
# robot = env.GetRobots()[0]

tuck_dof_values = None
left_tuck_dof_values = [0.32592421901965096, -2.34200605696553, 2.2443831554480718, 0.6143840852897609, 0.4095033675810835, 0.8291095083790417, -0.5451207814487959]
right_tuck_dof_values = [-0.3264493983797501, -2.337921596190979, -2.2443558819818707, 0.6147207056518011, -0.40258980361318003, 0.8277095613394793, 0.538388770740175]

env = openravepy.Environment()
env.SetViewer('qtcoin')
env.Load('../../test_domains/TorsenLSD/Environments/two_tables_parts.dae')
# # YuMiOpenRaveRobotModel(env, False)

openravepy.misc.DrawAxes(env, [1,0,0,0,0,0,0])
h = None
part_name = None

In [3]:
# Load initial transforms
part_init_transforms = {}
with open('../../test_domains/TorsenLSD/Environments/init_transforms.pkl') as f:
    part_init_transforms = pickle.load(f)

In [4]:
# FetchOpenRaveRobotModel(env, False)
# env.GetKinBody('fetch').SetTransform(part_init_transforms['fetch'])
fetch_tuck_dof_values = env.GetRobot('fetch').GetActiveDOFValues()

In [ ]:
# taskmanip = interfaces.TaskManipulation(env.GetKinBody('fetch'))
# taskmanip.ReleaseFingers()

In [ ]:
reference_env = Environment()
# reference_env.Load('../../test_domains/TorsenLSD/Environments/reference/lefthalf.dae')
# reference_env.Load('../../test_domains/TorsenLSD/Environments/reference/righthalf.dae')
reference_env.Load('../../test_domains/TorsenLSD/Environments/keva_test.dae')
reference_env.SetViewer('qtosg')

In [ ]:
print(reference_env.GetKinBody('table60').GetTransform())
print(env.GetKinBody('table60').GetTransform())
t1 = reference_env.GetKinBody('table60').GetTransform()
t2 = env.GetKinBody('table60').GetTransform()
print(t1[2,3] - t2[2,3])

In [ ]:
assembled_env = Environment()
assembled_env.Load('../../test_domains/TorsenLSD/Environments/reference/lefthalf_assembled.dae')
# assembled_env.Load('../../test_domains/TorsenLSD/Environments/reference/righthalf_assembled.dae')


In [32]:
def rotate_z(rot_angle):
    # rotate around z axis
    rotation_matrix = np.identity(4)
    rotation_matrix[0][0] = math.cos(rot_angle)
    rotation_matrix[0][1] = -(math.sin(rot_angle))
    rotation_matrix[1][0] = math.sin(rot_angle)
    rotation_matrix[1][1] = math.cos(rot_angle)
    return rotation_matrix

def rotate_y(rot_angle):
    # rotate around y axis
    rotation_matrix = np.identity(4)
    rotation_matrix[0][0] = math.cos(rot_angle)
    rotation_matrix[0][2] = math.sin(rot_angle)
    rotation_matrix[2][0] = -math.sin(rot_angle)
    rotation_matrix[2][2] = math.cos(rot_angle)
    return rotation_matrix

def rotate_x(rot_angle):
    # rotate around x axis
    rotation_matrix = np.identity(4)
    rotation_matrix[1][1] = math.cos(rot_angle)
    rotation_matrix[1][2] = -(math.sin(rot_angle))
    rotation_matrix[2][1] = math.sin(rot_angle)
    rotation_matrix[2][2] = math.cos(rot_angle)
    return rotation_matrix
    
def translate(x, y, z):
    # rotate around x axis
    rotation_matrix = np.identity(4)
    rotation_matrix[0][3] = x
    rotation_matrix[1][3] = y
    rotation_matrix[2][3] = z
    return rotation_matrix

In [33]:
def planToConfiguration(robot, goal_config):
    '''
    find motion plan to given IK
    goal_config: IK to move arm to
    '''

    # planner = ompl.OMPLPlanner('RRTConnect')
    planner = CBiRRTPlanner(timelimit=30.0)
    simplifier = ompl.OMPLSimplifier()
    # Motion Planning to reach joint state value(s)
    # Get trajectory from planner based on type of goal config passed
    # ( config a.k.a ik solutions a.k.a joint states )
    try:
        # if len(goal_configs) == 1:
            # If goal is a single IK solution, then call PlanToConfiguration
        trajectory_object = planner.PlanToConfiguration(robot, goal_config, smoothingitrs=30)
        # else:
            # If goal is a list of IK solutions, then call PlanToConfigurations
        # trajectory_object = planner.PlanToConfigurations(robot, goal_configs)
        if hasattr(planner, 'default_ompl_args'):
            print("simplifying..")
            # If planner is from OMPL, then simplify the trajectory
            trajectory_object = simplifier.ShortcutPath(robot,trajectory_object)
    except Exception as e:#PlanningError:
        print("Exception ", e)
        # print(__file__.split('/')[-1],": Could not find motion plan")
        return None
    print("retiming..")
    # Retime and serialize the trajectory
    _ = planningutils.RetimeTrajectory(trajectory_object)
    trajectory_object = trajectory_object.serialize()
    return trajectory_object

In [34]:
def get_housing_poses(side):
    pose_list = []

    if side.lower() == 'left':
        # grasping from protruding clamps where spur gears fit in
        z_rotation_matrix = rotate_z(np.pi)
        x_rotation_matrix = rotate_x(0)
        y_rotation_matrix = rotate_y(0)
        translate_matrix = translate(-0.23,0.053,-0.05)
        transformations = [z_rotation_matrix, translate_matrix]
    else:
        z_rotation_matrix = rotate_z(np.pi)
        x_rotation_matrix = rotate_x(0)
        y_rotation_matrix = rotate_y(0)
        translate_matrix = translate(-0.25,0.053,-0.05)
        transformations = [z_rotation_matrix, translate_matrix]
        
    grasp_pose = body_transform
    for transform in transformations:
        grasp_pose = np.matmul(grasp_pose, transform)
    
    pose_list.append(grasp_pose)
    return pose_list

def get_axle_poses():
    pose_list = []

    z_rotation_matrix = rotate_z(np.pi/2)
    x_rotation_matrix = rotate_x(np.pi/2)
    y_rotation_matrix = rotate_y(0)
    translate_matrix = translate(0,-0.02,-0.12)
    transformations = [x_rotation_matrix, z_rotation_matrix, translate_matrix]


    grasp_pose = body_transform
    for transform in transformations:
        grasp_pose = np.matmul(grasp_pose, transform)

    pose_list.append(grasp_pose)

    return pose_list


def get_wormspur_poses():
    pose_list = []

    z_rotation_matrix = rotate_z(0)
    x_rotation_matrix = rotate_x(0)
    y_rotation_matrix = rotate_y(-np.pi/2)
    translate_matrix = translate(0,0,-0.13)
    transformations = [y_rotation_matrix, z_rotation_matrix, translate_matrix]


    grasp_pose = body_transform
    for transform in transformations:
        grasp_pose = np.matmul(grasp_pose, transform)

    pose_list.append(grasp_pose)

    return pose_list

def get_upright_poses():
    pose_list = []
    
    z_rotation_matrix = rotate_z(-np.pi/2)
    x_rotation_matrix = rotate_x(0)
    y_rotation_matrix = rotate_y(0)
    translate_matrix = translate(-0.25,-0.012,0)
    transformations = [z_rotation_matrix, translate_matrix]
    
    grasp_pose = body_transform
    for transform in transformations:
        grasp_pose = np.matmul(grasp_pose, transform)
    
    pose_list.append(grasp_pose)

    return pose_list


def get_ring_gear_poses():
    pose_list = []

    z_rotation_matrix = rotate_z(0)
    x_rotation_matrix = rotate_x(np.pi/2)
    y_rotation_matrix = rotate_y(np.pi/2)
    translate_matrix = translate(-0.27,0,0)
    transformations = [y_rotation_matrix,x_rotation_matrix,translate_matrix]
    
    grasp_pose = body_transform
    for transform in transformations:
        grasp_pose = np.matmul(grasp_pose, transform)
    
    pose_list.append(grasp_pose)
    return pose_list

def get_drive_poses():
    pose_list = []

    z_rotation_matrix = rotate_z(0)
    x_rotation_matrix = rotate_x(np.pi/2)
    y_rotation_matrix = rotate_y(np.pi/2)
    translate_matrix = translate(-0.19,-0.0129,0)
    transformations = [y_rotation_matrix,x_rotation_matrix, translate_matrix]
        
    grasp_pose = body_transform
    for transform in transformations:
        grasp_pose = np.matmul(grasp_pose, transform)
    
    pose_list.append(grasp_pose)
    return pose_list

def get_shaft_poses():
    pose_list = []

    z_rotation_matrix = rotate_z(np.pi/2)
    x_rotation_matrix = rotate_x(0)
    y_rotation_matrix = rotate_y(-np.pi/2)
    translate_matrix = translate(0,-0.05,-0.12)
    transformations = [y_rotation_matrix, z_rotation_matrix, translate_matrix]
        
    grasp_pose = body_transform
    for transform in transformations:
        grasp_pose = np.matmul(grasp_pose, transform)

    pose_list.append(grasp_pose)
    return pose_list

def get_wheel_poses():
    pose_list = []

    z_rotation_matrix = rotate_z(0)
    x_rotation_matrix = rotate_x(0)
    y_rotation_matrix = rotate_y(np.pi)
    translate_matrix = translate(0,0,-0.16)
    transformations = [y_rotation_matrix, z_rotation_matrix, translate_matrix]

    grasp_pose = body_transform
    for transform in transformations:
        grasp_pose = np.matmul(grasp_pose, transform)

    pose_list.append(grasp_pose)
    return pose_list

def get_base_poses():
    pose_list = []

    com = env.GetKinBody('Base').ComputeAABB().pos()
    t = env.GetKinBody('Base').GetTransform()
    x_diff = abs(com[0] - t[0,3])
    y_diff = abs(com[1] - t[1,3])
    z_diff = abs(com[2] - t[2,3])

    z_rotation_matrix = rotate_z(0)
    x_rotation_matrix = rotate_x(0)
    y_rotation_matrix = rotate_y(np.pi/2)
    translate_matrix_com = translate(x_diff,y_diff,z_diff)
    translate_matrix = translate(-0.22,0,0)
    transformations = [translate_matrix_com, y_rotation_matrix, z_rotation_matrix, translate_matrix]

    grasp_pose = body_transform
    for transform in transformations:
        grasp_pose = np.matmul(grasp_pose, transform)

    pose_list.append(grasp_pose)
    return pose_list

In [35]:
def set_human_fixed_pose(base_name, part_name):
    print("base: ", base_name)
    print("part: ", part_name)
    body = env.GetKinBody(part_name)
    refw_T_base = assembled_env.GetKinBody(base_name).GetTransform()
    part_T_refw = assembled_env.GetKinBody(part_name).GetTransform()

    base_T_part = np.matmul(np.linalg.inv(refw_T_base), part_T_refw)

    w_T_base = env.GetKinBody(base_name).GetTransform()

    new_pose = np.matmul(w_T_base, base_T_part)

    body.SetTransform(new_pose)

In [36]:
def get_delta_trajectory(robot, end_pose, current_pose):
    planner = CBiRRTPlanner()
    difference_mat = end_pose - current_pose
    v_3D = difference_mat[:3,3]
    distance = np.sqrt(np.sum(v_3D**2))
    direction = list(v_3D/distance)

    try:
        trajectory_object = planner.PlanToEndEffectorOffset(robot, direction, distance,smoothingitrs=10,timelimit=10.)
        # trajectory_object = planner.PlanToEndEffectorOffset(robot, direction, distance)
    except PlanningError as e:
        print(__file__.split('/')[-1],": Could not find delta motion plan",e)
        return None
    _ = planningutils.RetimeTrajectory(trajectory_object)
    trajectory_object = trajectory_object.serialize()
    return trajectory_object

In [37]:
def execute_trajectory(robot, trajectory_object):
    # Deserialize the trajectory for execution
    des_traj = RaveCreateTrajectory(env, '')
    Trajectory.deserialize(des_traj, trajectory_object)
    print("waypoint count:", des_traj.GetNumWaypoints())
    # Execute on Openrave
    with robot:
        robot.GetController().SetPath(des_traj)
    robot.WaitForController(0)

In [38]:
def get_fast_ik_solutions(env, robot, end_effector_transform, do_check_collisions=False):
    filter_option = openravepy.IkFilterOptions.CheckEnvCollisions
    # filter_option = None
    if not do_check_collisions:
        filter_option = openravepy.IkFilterOptions.IgnoreEndEffectorCollisions
    
    ikmodel = openravepy.databases.inversekinematics.InverseKinematicsModel(robot, iktype=openravepy.IkParameterization.Type.Transform6D)
    if not ikmodel.load():
        raveLogInfo("Generating IKModel for " + str(robot))
        ikmodel.autogenerate()
    solutions = ikmodel.manip.FindIKSolutions(end_effector_transform, filter_option)
    # DrawAxes(self.env, end_effector_transform)
    print len(solutions), " IK solutions found" 

    return solutions[:20] # return only 20 solutions

In [ ]:
# def set_yumi_manipulator(side):
#     global sim, tuck_dof_values, left_tuck_dof_values, right_tuck_dof_values

#     if side == 'right':
#         robot.SetActiveManipulator('right_arm_effector')
#         right_arm_jointnames = ["yumi_joint_1_r", "yumi_joint_2_r", "yumi_joint_7_r", "yumi_joint_3_r",                                             "yumi_joint_4_r","yumi_joint_5_r", "yumi_joint_6_r"]
#         robot.SetActiveDOFs([robot.GetJoint(name).GetDOFIndex() for name in right_arm_jointnames])
#         taskmanip = interfaces.TaskManipulation(robot)
#         taskmanip.ReleaseFingers(movingdir=[1])
#         tuck_dof_values = right_tuck_dof_values[:]
#     else:
#         robot.SetActiveManipulator('left_arm_effector')
#         left_arm_jointnames = ["yumi_joint_1_l", "yumi_joint_2_l", "yumi_joint_7_l", "yumi_joint_3_l",                                              "yumi_joint_4_l", "yumi_joint_5_l", "yumi_joint_6_l"]
#         robot.SetActiveDOFs([robot.GetJoint(name).GetDOFIndex() for name in left_arm_jointnames])
#         taskmanip = interfaces.TaskManipulation(robot)
#         taskmanip.ReleaseFingers(movingdir=[1])
#         tuck_dof_values = left_tuck_dof_values[:]

#     sim.ik_solver = sim.set_IK_solver('world', str(robot.GetActiveManipulator().GetEndEffector().GetName()), sim.urdf_str)

# set_yumi_manipulator('left')

## TESTING MODELS

In [ ]:
sequence = -2
part_sequence = ['UprightClamp', 'RingGear', 'LeftHousing', 'WormSpur', 'WormSpur', 'WormSpur', 'Drive', 'Axle']
part_names = ['UprightClamp', 'RingGear', 'Housing', 'WormSpur1', 'WormSpur2', 'WormSpur3', 'Drive', 'Axle']
side = 'Right'
print(part_names[sequence])

# Load part into environment
model_dir = '../../test_domains/TorsenLSD/Environments/individual_centered/'
# model_file = model_dir + part_sequence[sequence] + '.dae'
model_file = model_dir + 'Base.dae'

part_model = env.ReadKinBodyURI(model_file)
env.AddKinBody(part_model)


In [ ]:
# part_name = side + part_names[sequence]
part_name = 'Base'
print(part_name)

part_model.SetName(part_name)
# if part_name in part_init_transforms.keys():
#     part_model.SetTransform(part_init_transforms[part_name])

In [ ]:
thermo_body1 = env.GetKinBody('fetchthermo2')
thermo_body2 = env.GetKinBody('thermoleft2')
thermo_body3 = env.GetKinBody('thermoright2')

for body in [thermo_body1, thermo_body2, thermo_body3]:
    current_transform = body.GetTransform()
    modified_pose = np.matmul(current_transform, translate(0,0,10))
    body.SetTransform(modified_pose)

In [ ]:
# Load initial transforms
clamp_transforms = {}
with open('../../test_domains/TorsenLSD/Environments/clamp_positions.pkl') as f:
    clamp_transforms = pickle.load(f)

In [ ]:
clamp_transforms['LeftUprightClamp'] = env.GetKinBody('LeftUprightClamp').GetTransform()

In [ ]:
# t = env.GetKinBody('table60').GetTransform()
body = env.GetKinBody('LeftUprightClamp')
body.SetTransform(clamp_transforms['LeftUprightClamp'])

In [ ]:
body = env.GetKinBody('RightUprightClamp')
t = body.GetTransform()
t[2,3] -= 0.02
# modified_pose = np.matmul(current_transform, translate(0,0,-0.02))
body.SetTransform(t)

In [ ]:
print(env.GetKinBody('table60').GetTransform())
print(env.GetKinBody('table61').GetTransform())

In [ ]:
for body in env.GetBodies():
    if body.GetName() not in ['yumi', 'fetch', 'table6']:
        t = body.GetTransform()
        t[2,3] -= 0.02
        # modified_pose = np.matmul(current_transform, translate(0,0,-0.02))
        body.SetTransform(t)

In [44]:
# Testing code

# Translate/Rotate part as required 
# body = env.GetKinBody(part_name)
body = env.GetKinBody('RightShaft3')
# body_dims = env.GetKinBody(part_name).ComputeAABB().extents()

current_transform = body.GetTransform()
print(current_transform)
# modified_pose = np.matmul(current_transform, rotate_x(-np.pi/2))
# modified_pose = np.matmul(current_transform, translate(0.1,0,0))
modified_pose = np.matmul(current_transform, translate(0,-0.01,0))
body.SetTransform(modified_pose)

# Set body name
# env.GetKinBody('table600').SetName('table61')

# Remove body
# env.Remove(env.GetKinBody(part_name))

# Set transform to another body
# t = env.GetKinBody('table60').GetTransform()
# body = env.GetKinBody(part_name)
# body.SetTransform(t)

# Save placed position transform
# part_init_transforms[part_name] = body.GetTransform()

# Set Saved transform
# env.GetKinBody('LeftAxle').SetTransform(part_init_transforms['LeftAxle'])


# Save all body transforms
# for body in env.GetBodies():
#     part_init_transforms[body.GetName()] = body.GetTransform()
# part_init_transforms

[[-1.00000000e+00  1.22464680e-16  1.23259513e-31  2.95229167e-01]
 [-1.22464680e-16 -1.00000000e+00  4.71027769e-16 -4.93881524e-01]
 [ 1.80943778e-31  4.71027769e-16  1.00000000e+00  1.44770722e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
for body in env.GetBodies():
    part_init_transforms[body.GetName()] = body.GetTransform()

In [ ]:
part_init_transforms['fetch'] = env.GetKinBody('fetch').GetTransform()

In [ ]:
# Save the init transforms
with open('../../test_domains/TorsenLSD/Environments/clamp_positions.pkl', 'wb') as f:
    pickle.dump(clamp_transforms, f)

## Pick pose


In [ ]:
side = 'left'

In [ ]:
# move the thermocol back in place
if robot.GetName() == 'fetch':
    thermo2_body = env.GetKinBody('fetchthermo2')
else:
    thermo2_body = env.GetKinBody('thermo{}2'.format(side))

current_transform = thermo2_body.GetTransform()
modified_pose = np.matmul(current_transform, translate(0,0,0.4))
thermo2_body.SetTransform(modified_pose)

In [ ]:
if robot.GetName() == 'fetch':
    robot.SetActiveDOFValues(fetch_tuck_dof_values)
else:
    set_yumi_manipulator(side)
    if side == 'right':
        robot.SetActiveDOFValues(right_tuck_dof_values)
    else:
        robot.SetActiveDOFValues(left_tuck_dof_values)
if h is not None:
    h.Close()

In [12]:
# robot = env.GetRobot('fetch')
robot = env.GetRobot('yumi')

In [ ]:
if robot.GetName() == 'fetch':
    robot.SetActiveDOFValues(fetch_tuck_dof_values)
else:
    robot.SetActiveDOFValues(left_tuck_dof_values)

In [27]:
part_name = 'LeftWheel'

In [28]:
# pick pose

if h is not None:
    h.Close()

body = env.GetKinBody(part_name)
body_transform = body.GetTransform()
body_dim = body.ComputeAABB().extents()*2

if 'Housing' in part_name:
    if 'Left' in part_name:
        grasp_poses = get_housing_poses('left')
    else:
        grasp_poses = get_housing_poses('right')
elif 'Axle' in part_name:
    grasp_poses = get_axle_poses()
elif 'Drive' in part_name:
    grasp_poses = get_drive_poses()
elif 'Upright' in part_name:
    grasp_poses = get_upright_poses()
elif 'WormSpur' in part_name:
    grasp_poses = get_wormspur_poses()
elif 'Shaft' in part_name:
    grasp_poses = get_shaft_poses()
elif 'Wheel' in part_name:
    grasp_poses = get_wheel_poses()
elif 'Base' in part_name:
    grasp_poses = get_base_poses()
elif 'RingGear' in part_name:
    grasp_poses = get_ring_gear_poses()
else:
    print("unknown part")

    # z_rotation_matrix = rotate_z(0)
    # x_rotation_matrix = rotate_x(np.pi/2)
    # y_rotation_matrix = rotate_y(np.pi/2)
    # translate_matrix = translate(-0.27,0,0)
    # transformations = [y_rotation_matrix,x_rotation_matrix,translate_matrix]
    
    # grasp_pose = body_transform
    # for transform in transformations:
    #     grasp_pose = np.matmul(grasp_pose, transform)
    
    grasp_poses = []
    grasp_poses.append(grasp_pose)

print(grasp_poses)
grasp_pose = grasp_poses[0]

h = misc.DrawAxes(env, grasp_pose)

[array([[ 2.22044605e-16, -1.00000000e+00, -2.71926215e-32,
         3.95229161e-01],
       [-1.00000000e+00, -2.22044605e-16,  3.67517926e-15,
         4.93881524e-01],
       [-3.67517926e-15,  0.00000000e+00, -1.00000000e+00,
         2.95370712e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])]


## PrePickPose

In [29]:
if robot.GetName() == 'fetch':
    pre_grasp_offset = 0.07
    pre_grasp_matrix = np.identity(4)
    pre_grasp_matrix[0][3] -= pre_grasp_offset
else: #yumi
    pre_grasp_offset = 0.04
    pre_grasp_matrix = np.identity(4)
    pre_grasp_matrix[2][3] -= pre_grasp_offset

prepickup_gripper_pose = np.matmul(grasp_pose, pre_grasp_matrix)

if h is not None:
    h.Close()
h = misc.DrawAxes(env, prepickup_gripper_pose)

## goto prepick


In [30]:
# Single arm IK
if robot.GetName() == 'fetch':
    ik_sols = get_fast_ik_solutions(env, env.GetRobot('fetch'), prepickup_gripper_pose, do_check_collisions=True)
else:
    ik_sols = sim.robots['yumi'].get_ik_solutions(prepickup_gripper_pose, check_collisions=True)
print(len(ik_sols))

11


In [ ]:
robot.SetActiveDOFValues(ik_sols[0])

In [ ]:
if robot.GetName() == 'fetch':
    robot.SetActiveDOFValues(fetch_tuck_dof_values)
else:
    robot.SetActiveDOFValues(left_tuck_dof_values)

In [ ]:
goal_configs = ik_sols

dof_values = robot.GetActiveDOFValues()

# Plan for the IKs
print(len(goal_configs))
for goal_config in goal_configs:
    i = 0
    while i < 5:
        robot.SetActiveDOFValues(dof_values)
        with env:
            trajectory_object = planToConfiguration(robot, goal_config)
        if trajectory_object is not None:
            break
        i += 1
    if trajectory_object is not None:
        print("found")
        break

robot.SetActiveDOFValues(dof_values)
# robot.SetActiveDOFValues(tuck_dof_values)

In [ ]:
execute_trajectory(robot, trajectory_object)

## delta trajectory to grasp pose

In [ ]:
if h is not None:
    h.Close()
h = misc.DrawAxes(env, grasp_pose)

trajectory_object = get_delta_trajectory(robot, grasp_pose, prepickup_gripper_pose)

In [ ]:
execute_trajectory(robot, trajectory_object)

In [ ]:
robot.Grab(env.GetKinBody(part_name))

In [ ]:
side = 'left'
if robot.GetName() == 'fetch':
    thermo2_body = env.GetKinBody('fetchthermo2')
else:
    thermo2_body = env.GetKinBody('thermo{}2'.format(side))

current_transform = thermo2_body.GetTransform()
modified_pose = np.matmul(current_transform, translate(0,0,-0.4))
thermo2_body.SetTransform(modified_pose)

## Post pickup pose

In [ ]:
# fetch
if robot.GetName() == 'fetch':
    post_grasp_offset = 0.07
    post_grasp_matrix = np.identity(4)
    post_grasp_matrix[0][3] -= post_grasp_offset
else:
    post_grasp_offset = 0.04
    post_grasp_matrix = np.identity(4)
    post_grasp_matrix[1][3] -= post_grasp_offset

postpickup_gripper_pose = np.matmul(grasp_pose, post_grasp_matrix)

if h is not None:
    h.Close()
h = misc.DrawAxes(env, postpickup_gripper_pose)

In [ ]:
dof_values = robot.GetActiveDOFValues()

## delta trajectory to post pickup

In [ ]:
trajectory_object = get_delta_trajectory(robot, postpickup_gripper_pose, grasp_pose)

In [ ]:
execute_trajectory(robot, trajectory_object)

## put down pose

In [ ]:
dof_values = robot.GetActiveDOFValues()

In [ ]:
# LeftUpright putdown pose
if h is not None:
    h.Close()

if part_name == 'LeftUprightClamp':
    pass
    # table_transform = env.GetKinBody('table60').GetTransform()

    # z_rotation_matrix = rotate_z(-np.pi/2)
    # y_rotation_matrix = rotate_y(np.pi)
    # x_rotation_matrix = rotate_x(0)
    # translate_matrix = translate(-.15,-0.05,-0.2575)

    # transformations = [y_rotation_matrix, x_rotation_matrix, z_rotation_matrix, translate_matrix]
    # putdown_gripper_pose = table_transform
    # for transform in transformations:
    #     putdown_gripper_pose = np.matmul(putdown_gripper_pose, transform)

    # # putdown_gripper_pose = table_transform

    # h = misc.DrawAxes(env, putdown_gripper_pose)

elif part_name == 'RightUprightClamp':
    pass
    # table_transform = env.GetKinBody('table60').GetTransform()

    # z_rotation_matrix = rotate_z(-np.pi/2)
    # y_rotation_matrix = rotate_y(np.pi)
    # x_rotation_matrix = rotate_x(0)
    # translate_matrix = translate(.15,-0.05,-0.2575)

    # transformations = [y_rotation_matrix, x_rotation_matrix, z_rotation_matrix, translate_matrix]
    # putdown_gripper_pose = table_transform
    # for transform in transformations:
    #     putdown_gripper_pose = np.matmul(putdown_gripper_pose, transform)

    # # putdown_gripper_pose = table_transform

    # h = misc.DrawAxes(env, putdown_gripper_pose)

else:
    # put down pose
    if side.lower() == 'right':
        reference_root_part = 'RightUprightClamp'
    else:
        reference_root_part = 'LeftUprightClamp'

    robot_world_transform = robot.GetLink('world').GetTransform() # w_T_r
    gripper_base_link = robot.GetActiveManipulator().GetEndEffector()
    gripper_base_transform = gripper_base_link.GetTransform() # w_T_g
    grabbed_object_pose = env.GetKinBody(part_name).GetTransform() # w_T_obj
    object_pose_wrt_gripper = np.matmul(np.linalg.inv(gripper_base_transform), grabbed_object_pose) # g_T_obj

    reference_root_transform = reference_env.GetKinBody(reference_root_part).GetTransform() # w_T_base
    reference_part_transform = reference_env.GetKinBody(part_name).GetTransform() # w_T_obj
    reference_part_wrt_root = np.matmul(np.linalg.inv(reference_root_transform),
                                                        reference_part_transform) # base_T_obj
    root_transform = env.GetKinBody(reference_root_part).GetTransform() # w_T_base

    updated_object_pose = np.matmul(root_transform, reference_part_wrt_root) # w_T_obj : new pose of object

    object_pose_wrt_robot = np.matmul(np.linalg.inv(robot_world_transform), updated_object_pose) # r_T_obj
    putdown_gripper_pose = np.matmul(object_pose_wrt_robot, np.linalg.inv(object_pose_wrt_gripper)) # r_T_g

    h = misc.DrawAxes(env, putdown_gripper_pose)

In [ ]:
# FETCH PUTDOWN POSE
reference_root_part = 'table60'

robot_world_transform = robot.GetLink('base_link').GetTransform() # w_T_r
gripper_base_link = robot.GetActiveManipulator().GetEndEffector()
gripper_base_transform = gripper_base_link.GetTransform() # w_T_g
grabbed_object_pose = robot.GetGrabbed()[0].GetTransform() # w_T_obj
object_pose_wrt_gripper = np.matmul(np.linalg.inv(gripper_base_transform), grabbed_object_pose) # g_T_obj

reference_root_transform = reference_env.GetKinBody(reference_root_part).GetTransform() # w_T_base
reference_part_transform = reference_env.GetKinBody(part_name).GetTransform() # w_T_obj
reference_part_wrt_root = np.matmul(np.linalg.inv(reference_root_transform),
                                                    reference_part_transform) # base_T_obj
root_transform = env.GetKinBody(reference_root_part).GetTransform() # w_T_base

updated_object_pose = np.matmul(root_transform, reference_part_wrt_root) # w_T_obj : new pose of object

object_pose_wrt_robot = np.matmul(np.linalg.inv(robot_world_transform), updated_object_pose) # r_T_obj
putdown_gripper_pose = np.matmul(object_pose_wrt_robot, np.linalg.inv(object_pose_wrt_gripper)) # r_T_g

putdown_gripper_pose = np.matmul(robot_world_transform, putdown_gripper_pose)


h = misc.DrawAxes(env, putdown_gripper_pose)

## Pre Put Down Pose

In [ ]:
# FETCH
if robot.GetName() == 'fetch':
    pre_putdown_offset = 0.07
else:
    pre_putdown_offset = 0.05

preputdown_gripper_pose = putdown_gripper_pose.copy()
preputdown_gripper_pose[2][3] += pre_putdown_offset

if h is not None:
    h.Close()
h = misc.DrawAxes(env, preputdown_gripper_pose)

## move to pre put down

In [ ]:
# Single arm IK
if robot.GetName() == 'fetch':
    ik_sols = get_fast_ik_solutions(env, env.GetRobot('fetch'), preputdown_gripper_pose, do_check_collisions=True)
else:
    ik_sols = sim.robots['yumi'].get_ik_solutions(preputdown_gripper_pose, check_collisions=True)
print(len(ik_sols))

In [ ]:
robot.SetActiveDOFValues(ik_sols[0])

In [ ]:
robot.SetActiveDOFValues(dof_values)

In [ ]:
goal_configs = ik_sols

dof_values = robot.GetActiveDOFValues()

# trajectories = {}
# Plan for the IKs
print(len(goal_configs))
for j, goal_config in enumerate(goal_configs):
    print(j)
    i = 0
    while i < 3:
        robot.SetActiveDOFValues(dof_values)
        with env:
            trajectory_object = planToConfiguration(robot, goal_config)
        if trajectory_object is not None:
            break
        i += 1
    if trajectory_object is not None:
        print("found")
        # trajectories[j] = trajectory_object
        break

robot.SetActiveDOFValues(dof_values)
# robot.SetActiveDOFValues(tuck_dof_values)

In [ ]:
execute_trajectory(robot, trajectory_object)

## delta trajectory to put down pose

In [ ]:
trajectory_object = get_delta_trajectory(robot, putdown_gripper_pose, preputdown_gripper_pose)

In [ ]:
execute_trajectory(robot, trajectory_object)

In [ ]:
robot.ReleaseAllGrabbed()

## post put down pose

In [ ]:
if robot.GetName() == 'fetch':
    # Set PostPickPose
    post_putdown_offset = 0.05
    postputdown_gripper_transform = np.identity(4)
    postputdown_gripper_transform[0][3] -= post_putdown_offset

    postputdown_gripper_pose = np.matmul(putdown_gripper_pose, postputdown_gripper_transform)
else:
    # Set PostPickPose
    post_putdown_offset = 0.04
    postputdown_gripper_transform = np.identity(4)
    postputdown_gripper_transform[2][3] -= post_putdown_offset

    postputdown_gripper_pose = np.matmul(putdown_gripper_pose, postputdown_gripper_transform)

if h is not None:
    h.Close()
h = misc.DrawAxes(env, postputdown_gripper_pose)

In [ ]:
trajectory_object = get_delta_trajectory(robot, postputdown_gripper_pose, putdown_gripper_pose)

In [ ]:
execute_trajectory(robot, trajectory_object)

In [ ]:
set_human_fixed_pose('LeftUprightClamp', part_name)

# OLD

In [ ]:
def create_plank(env,plankname,plank_transform=np.eye(4)):
    with env:
        body = RaveCreateKinBody(env,'')
        body.SetName(plankname)
        # body.InitFromBoxes(numpy.array([[0.,0.,0.,0.05884,0.01162,5*0.003875]]),True)
        # body.InitFromBoxes(numpy.array([[0.,0.,0.,0.4/2,0.3/2,0.0165/2]]),True) # thermoleft/right
        # body.InitFromBoxes(numpy.array([[0.,0.,0.,0.45/2,0.45,0.0165/2]]),True) # fetch table
        body.InitFromBoxes(numpy.array([[0.,0.,0.,0.5/2,0.5/2,0.75]]),True) # fetch table
        env.AddKinBody(body)
        body.SetTransform(plank_transform)
        return body

plank = create_plank(env, 'human')

In [ ]:
env.Remove(env.GetKinBody('yumi'))

In [ ]:
# Translate
for body in env.GetBodies():
    name = body.GetName()
    if name.startswith('Left') and name != part_name:
        # body = env.GetKinBody('LeftRingGear')
        current_transform = body.GetTransform()
        modified_pose = np.matmul(current_transform, translate(0, 0, -0.2))
        # h2 = misc.DrawAxes(env, putdown_gripper_pose)
        Tz = openravepy.matrixFromPose(modified_pose)
        body.SetTransform(Tz)

In [ ]:
parts = ['Axle', 'Base', 'Clamp', 'DriveGear', 'L-Housing', 'PinionGear', 'R-Housing', 'RingGear', 'Shaft', 'SpurGear', 'Upright', 'Wheel', 'WormWheel']
model_file = '/home/local/ASUAD/asrinet1/Downloads/TorsenLSD/{}.stl'.format(parts[12])
model_file = '/home/local/ASUAD/asrinet1/Downloads/TorsenLSD/SpurGear.stl'
# model_file = '/home/local/ASUAD/asrinet1/Downloads/3_components.dae'
# model_file = '/home/local/ASUAD/asrinet1/Downloads/base00225.stl'
part_model = env.ReadKinBodyURI(model_file)
env.AddKinBody(part_model)

In [ ]:
# create the dual-arm ik solver
dualsolver = misc.MultiManipIKSolver([robot.GetManipulator('left_arm_effector'), robot.GetManipulator('right_arm_effector')])
print(dualsolver.manips)

filter_option = IkFilterOptions.CheckEnvCollisions
with env:
    ikmodel = databases.inversekinematics.InverseKinematicsModel(robot, iktype=IkParameterization.Type.Transform6D)
    if not ikmodel.load():
        raveLogInfo("Generating IKModel for " + str(robot))
        ikmodel.autogenerate()

# left_ik_solver = IK('world', 'gripper_l_base', urdf_string=sim.urdf_str)
manip = robot.GetManipulator('left_arm_effector')
print(manip.GetIkSolver())
# manip.SetIkSolver(left_ik_solver)
IkParameterization.Type.

solutions = dualsolver.findMultiIKSolution(Tgrasps=[left_gripper_pose, right_gripper_pose],filteroptions=IkFilterOptions.CheckEnvCollisions)